In [13]:
import pandas as pd
import numpy as np
import datetime as dt
from math import floor

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

df = pd.read_csv('cleaned.csv', sep=",")
df_copy = df.copy()
df.head()

,Unnamed: 0,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1
1,1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0
2,2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0
3,3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0
4,4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0


In [14]:
#feature engineering 
df = df.set_index('ID')

In [15]:
df['Age'] = int(pd.datetime.now().year) - df['Year_Birth']
df_copy['Age'] = int(pd.datetime.now().year) - df_copy['Year_Birth']

In [16]:
print("Columns with string datatype are:")
for col in df.columns:
    if df[col].dtypes == object:
        print(col)

Columns with string datatype are:
Education
Marital_Status
Dt_Customer


In [17]:
df_Edu = pd.DataFrame(df['Education'].value_counts()).reset_index()
df_Edu.columns = ['Education', 'Count']

In [18]:
df['Education'] = np.where(df['Education'] == '2n Cycle', 'Master', df['Education'])

In [19]:
df_Edu = pd.DataFrame(df['Education'].value_counts()).reset_index()
df_Edu.columns = ['Education', 'Count']

fig = px.bar(df_Edu,
            x='Education',
            y='Count',
            color='Education')
fig.update_layout(width=800, height=400, title='Education ')
fig.show()

In [22]:
# df['h# df['hasKid'] = np.where((df['Kidhome'] + df['Teenhome'] > 0), 1, 0)
df['Kids'] = df['Kidhome'] + df['Teenhome']

df['Kids'].describe()

count    2240.000000
mean        0.950446
std         0.751803
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         3.000000
Name: Kids, dtype: float64

In [24]:
df_Mar = pd.DataFrame(df['Marital_Status'].value_counts()).reset_index()
df_Mar.columns = ['Marital_Status', 'Count']
df_Mar

,Marital_Status,Count
0,Married,864
1,Together,580
2,Single,480
3,Divorced,232
4,Widow,77
5,Alone,3
6,Absurd,2
7,YOLO,2


In [25]:
mar_stat = ['Single', 'Widow', 'Alone', 'Absurd', 'YOLO']
df['Marital_Status'] = np.where(df['Marital_Status'].isin(mar_stat), 'Single', df['Marital_Status'])
df['Marital_Status'] = np.where(df['Marital_Status'].isin(['Married', 'Together']), 'Relationship', 'Single')
df_Mar = pd.DataFrame(df['Marital_Status'].value_counts()).reset_index()
df_Mar.columns = ['Marital_Status', 'Count']
df_Mar

,Marital_Status,Count
0,Relationship,1444
1,Single,796


In [ ]:
df_Edu = pd.DataFrame(df['Marital_Status'].value_counts()).reset_index()
df_Edu.columns = ['Marital_Status', 'Count']

fig = px.bar(df_Edu,
            x='Marital_Status',
            y='Count',
            color='Marital_Status')
fig.update_layout(width=800, height=400, title='Education ')
fig.show()